In [1]:
ini_date = '04/01/23 12:00:00'
end_date = '06/01/23 13:00:00'

tt_date = '06/01/23 11:50:00'

time_step=15

target_folder="./time-data/evaluation/"



#binary_sensors=["rule-lunch"]

binary_sensors=["cubiertos", "fridge", "microwave", "ollas"]

tags=["1","2"]

colors={}
colors["1"]="red"
colors["2"]="blue"
colors["3"]="orange"
colors["4"]="green"
colors["5"]="pink"
colors["6"]="brown"



In [2]:
def bb_intersection_over_union(boxA, boxB):
    if(not(boxB[0]<boxA[0] and boxB[1]>boxA[1])):
        if(boxB[0]<boxA[0]):
            return bb_intersection_over_union(boxB,boxA)
        if(boxB[1]<boxA[1]):
            return bb_intersection_over_union(boxB,boxA)
        if(boxB[0] < boxA[0] or boxB[0] > boxA[2]):
            return 0
        if(boxB[1] < boxA[1] or boxB[1] > boxA[3]):
            return 0
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    print(boxA,boxB," iou:",[xA,yA,xB,yB])

    # compute the area of intersection rectangle
    interArea = (xB - xA) * (yB - yA)

    
    if (xA>xB):
        return 0
    if (yA>yB):
        return 0    
    
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    if (iou<0):
        return 0
    # return the intersection over union value
    return interArea
bb_intersection_over_union([10, 20, 30, 50],[5, 10, 11, 21] )

[5, 10, 11, 21] [10, 20, 30, 50]  iou: [10, 20, 11, 21]


1

In [3]:
 def get_iou(bb1, bb2):
    assert bb1[0] <= bb1[2]
    assert bb1[1] <= bb1[3]
    assert bb2[0] <= bb2[2]
    assert bb2[1] <= bb2[3]

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1[0], bb2[0])
    y_top = max(bb1[1], bb2[1])
    x_right = min(bb1[2], bb2[2])
    y_bottom = min(bb1[3], bb2[3])

    print(" iou:",[x_left,y_top,x_right,y_bottom])
    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    
    # compute the area of both AABBs
    bb1_area = (bb1[2] - bb1[0]) * (bb1[3] - bb1[1])
    bb2_area = (bb2[2] - bb2[0]) * (bb2[3] - bb2[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    #iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    print(" intersection_area/bb1_area:",intersection_area,bb1_area)
    if intersection_area==0 or bb1_area==0:
        return 1
    if(intersection_area==bb1_area):
        return 1
    iou = intersection_area / float(bb2_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou
get_iou([22, 25, 22, 26],[10, 20, 30, 50] )

 iou: [22, 25, 22, 26]
 intersection_area/bb1_area: 0 0


1

In [4]:
#get_iou([158, 544, 277, 403],[383, 371, 491, 451])

In [5]:
import numpy as np
from pandas import read_csv    

import time

from datetime import datetime

def day_time(ti):
    return int((int)(ti/(60*60*24)))

t0 = (int)(datetime.strptime(ini_date, '%d/%m/%y %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0))

tN = (int)(datetime.strptime(end_date, '%d/%m/%y %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN))


tt = (int)(datetime.strptime(tt_date, '%d/%m/%y %H:%M:%S').timestamp())
print (tt_date,"TT date is", tt_date, "day:",day_time(tt))

nT=(int)(round((tN-t0-1)/time_step))
print(nT)


def absT(tt):
    return (int)(tt*time_step+t0)

def relT(ti):
    return (int)((ti-t0)/time_step)


04/01/23 12:00:00 Init date is 1672830000 day: 19361
06/01/23 13:00:00 End date is 1673006400 day: 19363
06/01/23 11:50:00 TT date is 06/01/23 11:50:00 day: 19363
11760


In [6]:
def repaintXY(x,y):
    return int(x*0.08965-196.41+150), int(-y*0.1078+1077)

def repaintXXYY(x,y):
    return int((x+196.41-150)/0.08965), int((y-1077)/(-0.1078))


print(repaintXY(2280,5135))
print(repaintXY(2860,6730))
print(repaintXXYY(209,351))
#print(repaintXY(5300,6450))
#print(repaintXY(6600,5200))

(157, 523)
(209, 351)
(2848, 6734)


In [7]:
series=read_csv("./sensors-location.tsv", sep="\t",header=None)
print(series.values)
sensor_location={}
for d in series.values:
    for s in d[0].split(","):
        sensor_location[s]=[]
for d in series.values:
    for s in d[0].split(","):
        sensor_location[s].append([float(d[1]),float(d[2]),float(d[3]),float(d[4]),float(d[5])])
    
print(sensor_location)
    

[['cubiertos' 5200 6000 6200 6940 1.0]
 ['fridge' 2600 6700 3960 6000 1.0]
 ['fridge' 1965 6820 4000 5500 0.5]
 ['microwave' 5730 6600 7120 5830 1.0]
 ['microwave' 5780 7060 6920 5650 0.5]
 ['ollas' 6180 8180 6840 7300 1.0]
 ['ollas' 5770 8370 7220 6620 0.5]
 ['rule-cooking' 4840 8710 6300 7350 0.75]
 ['rule-cooking' 5200 8540 6500 7560 1.0]
 ['rule-lunch' 2100 8000 5000 6580 0.75]
 ['rule-lunch' 2570 7690 4780 6800 1.0]]
{'cubiertos': [[5200.0, 6000.0, 6200.0, 6940.0, 1.0]], 'fridge': [[2600.0, 6700.0, 3960.0, 6000.0, 1.0], [1965.0, 6820.0, 4000.0, 5500.0, 0.5]], 'microwave': [[5730.0, 6600.0, 7120.0, 5830.0, 1.0], [5780.0, 7060.0, 6920.0, 5650.0, 0.5]], 'ollas': [[6180.0, 8180.0, 6840.0, 7300.0, 1.0], [5770.0, 8370.0, 7220.0, 6620.0, 0.5]], 'rule-cooking': [[4840.0, 8710.0, 6300.0, 7350.0, 0.75], [5200.0, 8540.0, 6500.0, 7560.0, 1.0]], 'rule-lunch': [[2100.0, 8000.0, 5000.0, 6580.0, 0.75], [2570.0, 7690.0, 4780.0, 6800.0, 1.0]]}


In [8]:
sensor_activation={}
for sensor in binary_sensors:
    sensor_activation[sensor]=[]
    series=read_csv(target_folder+sensor+".tsv", sep="\t",parse_dates=True,header=None)
    sensor_activation[sensor]=  np.array(series.values[:,[1]])  
    print(sensor_activation[sensor].shape)


(11760, 1)
(11760, 1)
(11760, 1)
(11760, 1)


In [9]:
user_location={}
for tag in tags:
    user_location[tag]=[]
    series=read_csv(target_folder+tag+".tsv", sep="\t",parse_dates=True,header=None)
    user_location[tag]=  np.array(series.values[:,[1,2,3,4,5]])  
    print(user_location[tag].shape)
    
    f= open(target_folder+tag+".XY.tsv", "w")
    for d in np.array(series.values[:,[1,2,3,4,5,6]]):
        if( 'None' in d):
            f.write(d[5]+"\t"+d[0]+"\t"+d[1]+"\t"+d[2]+"\t"+d[3]+"\t"+d[4]+"\n")
            continue
        x1,y1=repaintXY(float(d[0]),float(d[1]))
        x2,y2=repaintXY(float(d[2]),float(d[3]))
        f.write(str(d[5])+"\t"+str(x1)+"\t"+str(y1)+"\t"+str(x2)+"\t"+str(y2)+"\t"+str(d[4])+"\n")
        
    f.close()
print(user_location["1"])


(11760, 5)
(11760, 5)
[['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']
 ...
 ['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']]


In [10]:
series=read_csv(target_folder+"/1.tsv", sep="\t",parse_dates=True,header=None)
ts=  np.array(series.values[:,[5]])  
print(ts)
print(ts.shape[0])

[['None']
 ['None']
 ['None']
 ...
 ['None']
 ['None']
 ['None']]
11760


In [11]:
from os import listdir
from os.path import isfile, join

pathcamera1="../data-amaltea/fuzzy1C/"
filecamera1= [f for f in listdir(pathcamera1) if isfile(join(pathcamera1, f))]
filecamera1.sort()
pathcamera2="../data-amaltea/fuzzy2C/"
filecamera2= [f for f in listdir(pathcamera2) if isfile(join(pathcamera2, f))]
filecamera2.sort()
           

In [12]:
import cv2
import numpy as np
def findCloserImage(ti, pathcamera, startI=0):
    bef=False
    for idx,f in enumerate(pathcamera):
        if(idx<startI):
            continue
        t2=int(f[0:f.index(".")])
        #print(t2,ti,t2<=ti)
        if(t2<=ti):
            bef=True
        else:
            if bef:
                return f, idx
    return None, 0
      
    
#file1,_=findCloserImage(1671545535,filecamera1)
#file2,_=findCloserImage(1671545535,filecamera2)

#print(pathcamera1+file1)            
#print(pathcamera2+file2)  

#im1 = cv2.imread(pathcamera1+file1)
#im2 = cv2.imread(pathcamera2+file2)


#for file1 in filecamera1:
#    print(file1)
#    im1 = cv2.imread(pathcamera1+file1)
#    cv2.imshow('cam',im1)
#    cv2.waitKey(1)        



In [13]:
def intervals(s):
    ret=[]
    lastV=0
    lastI=-1
    for t in range(0,len(s)):
        if(lastV==1 and s[t]==0):
            print("\t interval ",t)
            ret.append((lastI,t-1))
        if(lastV==0 and s[t]==1):
            lastI=t
        lastV=s[t]
    print(ret)
    return ret

In [14]:
S=np.zeros((len(binary_sensors), nT))
for i,sensor in enumerate(binary_sensors):
    series=read_csv(target_folder+sensor+".tsv", sep="\t",parse_dates=True,header=None)
    S[i]=  np.array(series.values[:,[1]]).flatten()  
    print(S[i].shape)
print(S.shape)
print(np.where( (S[0]==1)))


(11760,)
(11760,)
(11760,)
(11760,)
(4, 11760)
(array([  212,   217,   337,   338,   375,   376,   382,   467,   468,
         492,   495,  1418,  2217,  2218,  2237,  2244,  2246,  2247,
        2250,  2335,  2336,  2823,  2844,  2845,  4771,  4819,  4869,
        4921,  4922,  5201,  5202,  5226,  5227,  6213,  6218,  6220,
        6243,  6244,  6287,  6288,  6289,  6301,  6336,  6337,  6384,
        6385,  6389,  6391,  6392,  6404,  6405,  6517,  6518,  6519,
        6523,  6525,  6553,  6554,  6555,  6559,  6560,  6574,  6578,
        7337,  7951,  7952,  7963,  7972,  7973,  8195,  8206,  8221,
        8222,  8516,  8682, 10845, 10948, 10949, 11016, 11017, 11045,
       11046, 11093, 11273, 11284, 11285, 11510, 11518, 11545, 11569,
       11589, 11663, 11667, 11685], dtype=int64),)


In [15]:
for i,sensor in enumerate(binary_sensors):
        print("Evaluating sensor activation of ",sensor, intervals(S[i]))
        for ii in intervals(S[i]):
            print("\t Interval",ii, (ii[0]), (ii[1]))


	 interval  213
	 interval  218
	 interval  339
	 interval  377
	 interval  383
	 interval  469
	 interval  493
	 interval  496
	 interval  1419
	 interval  2219
	 interval  2238
	 interval  2245
	 interval  2248
	 interval  2251
	 interval  2337
	 interval  2824
	 interval  2846
	 interval  4772
	 interval  4820
	 interval  4870
	 interval  4923
	 interval  5203
	 interval  5228
	 interval  6214
	 interval  6219
	 interval  6221
	 interval  6245
	 interval  6290
	 interval  6302
	 interval  6338
	 interval  6386
	 interval  6390
	 interval  6393
	 interval  6406
	 interval  6520
	 interval  6524
	 interval  6526
	 interval  6556
	 interval  6561
	 interval  6575
	 interval  6579
	 interval  7338
	 interval  7953
	 interval  7964
	 interval  7974
	 interval  8196
	 interval  8207
	 interval  8223
	 interval  8517
	 interval  8683
	 interval  10846
	 interval  10950
	 interval  11018
	 interval  11047
	 interval  11094
	 interval  11274
	 interval  11286
	 interval  11511
	 interval  11

In [16]:
from tkinter import *
from PIL import ImageTk, Image
from pandas import read_csv
import time
from threading import Thread
import threading

speed=0.05



incT=time_step

wait_step2step=False
wait_event=True


con = threading.Condition()

do_capture=True


lastIm1=0
lastIm2=0

images=[]

def create_rectangle(x1, y1, x2, y2, **kwargs):
    if(x1>x2):
        aux=x2
        x2=x1
        x1=aux
    if(y1>y2):
        aux=y2
        y2=y1
        y1=aux
        
    if 'alpha' in kwargs:
        alpha = int(kwargs.pop('alpha') * 255)
        fill = kwargs.pop('fill')
        fill = win.winfo_rgb(fill) + (alpha,)
        image = Image.new('RGBA', (x2-x1, y2-y1), fill)
        images.append(ImageTk.PhotoImage(image))
        canvas.create_image(x1, y1, image=ImageTk.PhotoImage(image), anchor='nw')
    canvas.create_rectangle(x1, y1, x2, y2, **kwargs)
    
def create_text(canvas, text): #center coordinates, radius
    play=">" if incT>0 else "<"
    if(not wait_step2step):
        play=play+play
        
    if(not wait_event):
        play=play+play
        
    
    canvas.create_rectangle(
    5, 5, 200, 20,
    outline="#fb0",
    fill="#fb0")
    canvas.create_text(100,10,fill="darkblue",text="["+play+"]"+text)

def create_big_text(canvas, text, index, back): #center coordinates, radius
    if(back):
        canvas.create_rectangle(
        20, 25, 400, 200,
        outline="#fb0",
        fill="#fb0")
    canvas.create_text(150,50+index*25,fill="darkblue",text=text)    

def create_circle(x, y, r, canvas, color): #center coordinates, radius
    x0 = x - r
    y0 = y - r
    x1 = x + r
    y1 = y + r
    canvas.create_oval(x0, y0, x1, y1, fill=color)

def create_bb(x1, y1, x2, y2, canvas, color, text, alpha=.5): #center coordinates, radius
    create_rectangle(
    x1, y1, x2, y2,    outline=color,
    fill=color, alpha=alpha)
    canvas.create_text((x1+x2)/2,(y1+y2)/2,fill="black",text=text)

def paintUserLocation(tt):
    t=relT(tt)
    for tag in tags:
        if( 'None' in user_location[tag][t]):
            continue

        user_location[tag][t]=user_location[tag][t].astype(np.float)
        x1,y1,x2,y2,w=user_location[tag][t][0],user_location[tag][t][1],user_location[tag][t][2],user_location[tag][t][3],user_location[tag][t][4]
        print("User@paintUserLocation",datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"), " x1,y1,x2,y2:",x1,y1,x2,y2)
        x1,y1=repaintXY(x1,y1)
        x2,y2=repaintXY(x2,y2)
        print("User#paintUserLocation",datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"), " x1,y1,x2,y2:",x1,y1,x2,y2)
        create_bb(x1,y1, x2,y2, canvas, colors[tag], tag+"("+str(t)+")",0.5)
    time.sleep(speed)
        
        
def paintSensorActivation(tt):
    t=relT(tt)
    for sensor in binary_sensors:
        if sensor_activation[sensor][t] > 0:
            #limpiamos
            canvas.create_image(400,300, image=img)
            create_text(canvas, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"))
            paintUserLocation(tt)
            #paintUserLocation(tt+time_step)
            #paintUserLocation(tt-time_step)
            #ponemos
            
            for sl in sensor_location[sensor]:
                x1,y1,x2,y2,w=sl[0],sl[1],sl[2],sl[3],sl[4]

                print("Sensor@paintSensorActivation", sensor, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"), " x1,y1,x2,y2,w:",x1,y1,x2,y2,w)
                x1,y1=repaintXY(x1,y1)
                x2,y2=repaintXY(x2,y2)                
                print("Sensor#paintSensorActivation", sensor, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"), " x1,y1,x2,y2,w:",x1,y1,x2,y2,w)
                create_bb(x1,y1, x2,y2, canvas, "#fb0",sensor,w*0.5)

            IoU(tt,sensor)
            #nb = input('Press to continue.. ')
            if(wait_event):
                con.acquire()
                con.wait()
                con.release()
            
def paintAllSensors():
    global do_capture
    for sensor in binary_sensors:
            for sl in sensor_location[sensor]:
                x1,y1,x2,y2,w=sl[0],sl[1],sl[2],sl[3],sl[4]

                print("Sensor@paintSensorActivation", sensor, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"), " x1,y1,x2,y2,w:",x1,y1,x2,y2,w)
                x1,y1=repaintXY(x1,y1)
                x2,y2=repaintXY(x2,y2)                
                print("Sensor#paintSensorActivation", sensor, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"), " x1,y1,x2,y2,w:",x1,y1,x2,y2,w)
                create_bb(x1,y1, x2,y2, canvas, "#fb0",sensor,w*0.5)

                
    con.acquire()
    con.wait()
    do_capture=False
    con.release()            

def IoU(tt,sensor):
    t=relT(tt)
    iou={}
    for ixtag,tag in enumerate(tags):
        if( 'None' in user_location[tag][t]):
            continue
        
        user_location[tag][t]=user_location[tag][t].astype(np.float)
        x1,y1,x2,y2,w=user_location[tag][t][0],user_location[tag][t][1],user_location[tag][t][2],user_location[tag][t][3],user_location[tag][t][4] 

        x1,y1=repaintXY(x1,y1)
        x2,y2=repaintXY(x2,y2) 

        iou[tag]=0
        for sl in sensor_location[sensor]:
            x1o,y1o,x2o,y2o,wo=sl[0],sl[1],sl[2],sl[3],sl[4]
            x1o,y1o=repaintXY(x1o,y1o)
            x2o,y2o=repaintXY(x2o,y2o)
            print("IoU#",[x1,y1,x2,y2],[x1o,y1o,x2o,y2o])
            if(x1o>x2o):
                aux=x2o
                x2o=x1o
                x1o=aux
            if(y1o>y2o):
                aux=y2o
                y2o=y1o
                y1o=aux
            if(x1>x2):
                aux=x2
                x2=x1
                x1=aux
            if(y1>y2):
                aux=y2
                y2=y1
                y1=aux
            print("IoU#",[x1,y1,x2,y2],[x1o,y1o,x2o,y2o],)
            
            viou=get_iou([x1,y1,x2,y2],[x1o,y1o,x2o,y2o])
            print("IoU:",tag,viou)
            print(w,wo,"F(IoU):",tag,viou*w*wo)
            iou[tag]=max(iou[tag],viou*w*wo)
            
        print("iou[tag]:",iou[tag])
        #if(iou[tag]>0):
        create_big_text(canvas, "Usuario "+str(tag)+" y S:"+str(sensor)+" w:"+str(iou[tag])+" ? ",ixtag, ixtag==0)


def paintSensorArea():
    for sensor in binary_sensors:
            for sl in sensor_location[sensor]:
                x1,y1,x2,y2,w=sl[0],sl[1],sl[2],sl[3],sl[4]

                print("Sensor@paintSensorArea", sensor, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"), " x1,y1,x2,y2,w:",x1,y1,x2,y2,w)

                x1,y1=repaintXY(x1,y1)
                x2,y2=repaintXY(x2,y2)                
                create_bb(x1,y1, x2,y2, canvas, "#fb0",sensor,w*0.5)
                print("Sensor#paintSensorArea", sensor, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"), " x1,y1,x2,y2,w:",x1,y1,x2,y2,w)

            con.acquire()
            con.wait()
            con.release()            

def paint(tt):
    t=relT(tt)
    create_text(canvas, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"))

    paintUserLocation(tt)
    paintSensorActivation(tt)    
        
def print0():
    
    global lastIm1
    global lastIm2
    
    paintAllSensors()
    nextStep()
    

def nextStep():
    global tt
    global lastIm1
    global lastIm2
    
    S=np.zeros((len(binary_sensors), nT))
    for i,sensor in enumerate(binary_sensors):
        series=read_csv(target_folder+sensor+".tsv", sep="\t",parse_dates=True,header=None)
        S[i]=  np.array(series.values[:,[1]]).flatten()  
        print(S[i].shape)
    print(S.shape)
    print(np.where( (S[0]==1)))
    
    
    for ix,sensor in enumerate(binary_sensors):
        print("Evaluating sensor activation of ",sensor, intervals(S[ix]))
        print(ix, S[ix], np.where( (S[ix]==1)))
        inter=intervals(S[ix])
        
        #for jx in range(0,len(inter)):
        jx=0
        while True:
    #    while True:
    
            if(wait_step2step and wait_event):
                tt=tt+incT
            else:
                if(incT>0):
                    jx=jx+1
                else:
                    jx=jx-1
                ii=inter[jx]
                print("\t Interval",ii, (ii[0]), (ii[1])) 
                tt=absT(ii[0])
        
            print("nextStep",tt,tN,t0,incT)
            if(jx>len(inter) or jx<0):
                break
            if(tt>tN or tt<t0):
                break


            t=relT(tt)
            if(t%10==0 or (wait_step2step and wait_event)):
                canvas.delete("all")
                canvas.create_rectangle(0, 0, 800, 800,fill="white")
                canvas.create_image(400, 300, image=img)
            paint(tt)



            file1,lastIm1=findCloserImage(tt,filecamera1,-1)
            file2,lastIm2=findCloserImage(tt,filecamera2,-1)
            print("images:",tt,file1,file2)
            if(file1 is not None and file2 is not None):
                im1 = cv2.imread(pathcamera1+file1)
                cv2.putText(im1, str(tt)+"#"+str(file1), (5,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
                im2 = cv2.imread(pathcamera2+file2)
                cv2.putText(im2, str(tt)+"#"+str(file2), (5,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)

                im1 = cv2.resize(im1, None, fx = 0.5, fy = 0.5, interpolation = cv2.INTER_CUBIC)
                im2 = cv2.resize(im2, None, fx = 0.5, fy = 0.5, interpolation = cv2.INTER_CUBIC)

                hor = np.concatenate((im1, im2), axis=1)
                cv2.imshow('cam',hor)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            if(wait_step2step):
                con.acquire()
                con.wait()
                con.release()



def callback(event):
    print( "clicked at", event.x, event.y)
    print( " world x,y at", repaintXXYY(event.x, event.y))
    xx,yy=repaintXXYY(event.x, event.y)
    f = open("./gt-labelling-sensors/user2.xy.tsv", "a")
    f.write(str(tt)+"\t"+datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")+"\t"+str(xx)+"\t"+str(yy)+"\n")
    f.close()
    #con.acquire()
    #con.notify()
    #con.release()

    
def motion(event):
    global do_capture
    if do_capture:
        x, y = event.x, event.y
        print('{}, {}'.format(x, y))    
        print(repaintXXYY(x, y))
        create_text(canvas, str(repaintXXYY(x, y)))
        
def key(event):
    print ("pressed", repr(event.char), str(repr(event.char)))   
    global incT
    global wait_step2step
    global wait_event
    
    if(str(repr(event.char)) == "'a'"):
        print('a!')
        incT=-time_step
        wait_step2step=False
        wait_event=False
    if(repr(event.char) == "'s'"):
        print('s!')
        incT=-time_step
        wait_step2step=False
        wait_event=True
    if(repr(event.char) == "'d'"):
        print('d!')
        incT=-time_step
        wait_step2step=True
        wait_event=True   
    if(repr(event.char) == "'l'"):
        print('l!')
        incT=time_step
        wait_step2step=False
        wait_event=False
    if(repr(event.char) == "'k'"):
        print('k!')
        incT=time_step
        wait_step2step=False
        wait_event=True
    if(repr(event.char) == "'j'"):
        print('j!')
        incT=time_step
        wait_step2step=True
        wait_event=True    
    create_text(canvas, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"))
    con.acquire()
    con.notify()
    con.release()
        
#Create an instance of tkinter window
win = Tk()

# Define the geometry of the window
#win.geometry("1010x612")

#frame = Frame(win, width=600, height=400)
#frame.pack()
#frame.place(anchor='center', relx=0.5, rely=0.5)

# Create an object of tkinter ImageTk
img = ImageTk.PhotoImage(Image.open("plano5.png"))

# Create a Label Widget to display the text or Image
#label = Label(frame, image = img)

#label.pack()
#label.bind('<Button 1>', motion)

canvas = Canvas(win, height=600, width=700)
canvas.create_image(400, 300, image=img)
canvas.bind("<Button-1>", callback)
win.bind("<Key>", key)
win.bind("<Motion>", motion)

#print0() 

canvas.pack()
print("pre loop")



thread = Thread(target = print0, args = ())
thread.setDaemon(True)
thread.start()



    
win.mainloop()


    




pre loop
Sensor@paintSensorActivation cubiertos 2023-01-06 11:50:00  x1,y1,x2,y2,w: 5200.0 6000.0 6200.0 6940.0 1.0
Sensor#paintSensorActivation cubiertos 2023-01-06 11:50:00  x1,y1,x2,y2,w: 419 430 509 328 1.0
Sensor@paintSensorActivation fridge 2023-01-06 11:50:00  x1,y1,x2,y2,w: 2600.0 6700.0 3960.0 6000.0 1.0
Sensor#paintSensorActivation fridge 2023-01-06 11:50:00  x1,y1,x2,y2,w: 186 354 308 430 1.0
Sensor@paintSensorActivation fridge 2023-01-06 11:50:00  x1,y1,x2,y2,w: 1965.0 6820.0 4000.0 5500.0 0.5
Sensor#paintSensorActivation fridge 2023-01-06 11:50:00  x1,y1,x2,y2,w: 129 341 312 484 0.5
Sensor@paintSensorActivation microwave 2023-01-06 11:50:00  x1,y1,x2,y2,w: 5730.0 6600.0 7120.0 5830.0 1.0
Sensor#paintSensorActivation microwave 2023-01-06 11:50:00  x1,y1,x2,y2,w: 467 365 591 448 1.0
Sensor@paintSensorActivation microwave 2023-01-06 11:50:00  x1,y1,x2,y2,w: 5780.0 7060.0 6920.0 5650.0 0.5
Sensor#paintSensorActivation microwave 2023-01-06 11:50:00  x1,y1,x2,y2,w: 471 315 573 4

442, 401
(5447, 6270)
430, 384
(5314, 6428)
419, 372
(5191, 6539)
382, 342
(4778, 6818)
321, 309
(4098, 7124)
285, 295
(3696, 7254)
228, 279
(3060, 7402)
198, 275
(2726, 7439)
190, 275
(2637, 7439)
183, 276
(2558, 7430)
182, 277
(2547, 7421)
177, 282
(2492, 7374)
170, 290
(2413, 7300)
164, 296
(2347, 7244)
153, 312
(2224, 7096)
147, 320
(2157, 7022)
138, 337
(2056, 6864)
129, 354
(1956, 6706)
127, 362
(1934, 6632)
126, 385
(1923, 6419)
135, 413
(2023, 6159)
148, 436
(2168, 5946)
167, 457
(2380, 5751)
181, 467
(2536, 5658)
217, 479
(2938, 5547)
254, 480
(3350, 5538)
272, 480
(3551, 5538)
300, 478
(3864, 5556)
322, 471
(4109, 5621)
335, 459
(4254, 5732)
338, 450
(4287, 5816)
339, 433
(4299, 5974)
334, 415
(4243, 6141)
331, 408
(4209, 6205)
320, 393
(4087, 6345)
301, 378
(3875, 6484)
279, 369
(3629, 6567)
256, 363
(3373, 6623)
235, 361
(3138, 6641)
216, 360
(2927, 6651)
203, 363
(2782, 6623)
197, 366
(2715, 6595)
190, 374
(2637, 6521)
185, 391
(2581, 6363)
185, 407
(2581, 6215)
191, 422
(